<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/ark_12112020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
!pip install -qq transformers
!pip install twokenize

In [88]:
import twokenize
twokenize.tokenize("lol ly x0x0,:D")

['lol', 'ly', 'x0x0', ',', ':D']

In [89]:
import torch
import numpy as np
import pandas as pd

In [90]:
df = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')

df.head()

,tweet,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [91]:
possible_labels=df.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0}

In [92]:
df['a_label']=df.subtask_a.replace(label_dict)
df.head()
df.head()

,tweet,subtask_a,subtask_b,subtask_c,a_label
id,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,0
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,0
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,1
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,0
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,1


In [93]:
from sklearn.model_selection import train_test_split
RANDOM_SEED=17

X_train,X_val,y_train,y_val = train_test_split(
  df.index.values,
  df.a_label.values,
  test_size=0.2,
  random_state=RANDOM_SEED,
  stratify=df.a_label.values
)

In [94]:
df['data_type']=['not_set']*df.shape[0]

df.loc[X_train,'data_type']='train'
# df.loc[df_val,'data_type']='val'
df.loc[X_val,'data_type']='val'
# df.loc[X_test,'data_type']='test'
df.head()

,tweet,subtask_a,subtask_b,subtask_c,a_label,data_type
id,,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,0,train
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,0,val
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,1,train
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,0,train
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,1,train


In [95]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-cased')

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [96]:
df['ark_sweet']=df.tweet.apply(twokenize.tokenize)


# encoder_data_train=tokenizer.batch_encode_plus(
#     df[df.data_type=='train'].tweet.values,
#     add_special_tokens=True,
#     return_attention_mask=True,
#     truncation=True,
#     padding=True,
#     max_length=256,
#     return_tensors='pt'
# )

# encoder_data_val=tokenizer.batch_encode_plus(
#     df[df.data_type=='val'].tweet.values,
#     add_special_tokens=True,
#     return_attention_mask=True,
#     truncation =True,
#     padding=True,
#     max_length=256,
#     return_tensors='pt'
# )


In [97]:
df.head()

,tweet,subtask_a,subtask_b,subtask_c,a_label,data_type,ark_sweet
id,,,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,0,train,"[@USER, She, should, ask, a, few, native, Amer..."
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,0,val,"[@USER, @USER, Go, home, you’re, drunk, !!!, @..."
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,1,train,"[Amazon, is, investigating, Chinese, employees..."
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,0,train,"[@USER, Someone, should'veTaken, "", this, piec..."
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,1,train,"[@USER, @USER, Obama, wanted, liberals, &, amp..."


In [98]:


train_labels=df[df.data_type=='train'].a_label.values
train_tokenized_sents=[]
for sent in df[df.data_type=='train'].tweet.values:
  se=['[CLS]']
  se.extend(twokenize.tokenize(sent))
  se.append('[SEP]') 
  train_tokenized_sents.append(se)

print("tokenized的第一句话:",train_tokenized_sents[0])
type(train_tokenized_sents)

tokenized的第一句话: ['[CLS]', '@USER', 'She', 'should', 'ask', 'a', 'few', 'native', 'Americans', 'what', 'their', 'take', 'on', 'this', 'is', '.', '[SEP]']


list

In [99]:
MAX_LEN=128

#将分割后的句子转化成数字  word-->idx
train_input_ids=[tokenizer.convert_tokens_to_ids(sent) for sent in train_tokenized_sents]
# print("转化后的第一个句子:",input_ids[0])

In [100]:
import keras
train_input_ids=keras.preprocessing.sequence.pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# print("Padding 第一个句子:",train_input_ids[0])

In [101]:
#建立mask
train_attention_masks = []
for seq in train_input_ids:
  seq_mask = [int(i>0) for i in seq]
  train_attention_masks.append(seq_mask)
# print("第一个attention mask:",train_attention_masks[0])

In [113]:
val_labels=df[df.data_type=='val'].a_label.values
val_tokenized_sents=[]
for sent in df[df.data_type=='val'].tweet.values:
  se=['[CLS]']
  se.extend(twokenize.tokenize(sent))
  se.append('[SEP]') 
  val_tokenized_sents.append(se)

MAX_LEN=128

#将分割后的句子转化成数字  word-->idx
val_input_ids=[tokenizer.convert_tokens_to_ids(sent) for sent in val_tokenized_sents]
val_input_ids=keras.preprocessing.sequence.pad_sequences(val_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
val_attention_masks = []
for seq in val_input_ids:
  seq_mask = [int(i>0) for i in seq]
  val_attention_masks.append(seq_mask)

In [114]:
torch.tensor(val_labels).size()

torch.Size([2648])

In [115]:
torch.tensor(val_attention_masks).size()

torch.Size([2648, 128])

In [116]:
from torch.utils.data import TensorDataset
dataset_train=TensorDataset(torch.tensor(train_input_ids),
              torch.tensor(train_attention_masks), 
              torch.tensor(train_labels))
dataset_val=TensorDataset(torch.tensor(val_input_ids),
              torch.tensor(val_attention_masks), 
              torch.tensor(val_labels))

In [84]:
df[df.data_type=='train'].a_label.values.shape

(13240,)

In [117]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE=16

dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=BATCH_SIZE
)
dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=BATCH_SIZE
)

In [118]:
from transformers import BertForSequenceClassification

In [119]:
model=BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [120]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
epochs = 8

scheduler = get_linear_schedule_with_warmup(optimizer, 
                        num_warmup_steps=0,
                        num_training_steps=len(dataloader_train)*epochs)

In [121]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def classification_report_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return classification_report(labels_flat, preds_flat)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [122]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':batch[0],
              'attention_mask':batch[1],
              'labels':batch[2],
              }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [123]:
epochs=3

In [124]:
from tqdm.notebook import tqdm
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.5368014697176813
Validation loss: 0.4840763191052948
F1 Score (Weighted): 0.7551020682352453



Epoch 2
Training loss: 0.43564000130374986
Validation loss: 0.4820899951709322
F1 Score (Weighted): 0.7714984231802313



Epoch 3
Training loss: 0.34596206749201364
Validation loss: 0.552885428026139
F1 Score (Weighted): 0.7515524548688481



In [126]:
df_sweet = pd.read_csv("testset-levela.tsv", sep='\t', header=0, index_col='id')
df_sweet.head()

,tweet
id,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
27014,"#ConstitutionDay is revered by Conservatives, ..."
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
13876,#Watching #Boomer getting the news that she is...
60133,#NoPasaran: Unity demo to oppose the far-right...


In [127]:
df_l = pd.read_csv("labels-levela.csv",header=None,names=['id','subtask_a'],index_col='id')
df_l.head()

,subtask_a
id,
15923,OFF
27014,NOT
30530,NOT
13876,NOT
60133,OFF


In [128]:
test = pd.merge(df_sweet, df_l, on=['id'])

In [130]:
possible_labels=test.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0}

In [131]:
test['a_label']=test.subtask_a.replace(label_dict)
test.head()

,tweet,subtask_a,a_label
id,,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,0
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,1
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,1
13876,#Watching #Boomer getting the news that she is...,NOT,1
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,0


In [132]:
test['data_type']=['test']*test.shape[0]

In [133]:
test_labels=df[df.data_type=='test'].a_label.values
test_tokenized_sents=[]
for sent in df[df.data_type=='test'].tweet.values:
  se=['[CLS]']
  se.extend(twokenize.tokenize(sent))
  se.append('[SEP]') 
  test_tokenized_sents.append(se)

MAX_LEN=128

#将分割后的句子转化成数字  word-->idx
test_input_ids=[tokenizer.convert_tokens_to_ids(sent) for sent in test_tokenized_sents]
test_input_ids=keras.preprocessing.sequence.pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
test_attention_masks = []
for seq in val_input_ids:
  seq_mask = [int(i>0) for i in seq]
  test_attention_masks.append(seq_mask)

In [137]:
dataset_test=TensorDataset(torch.tensor(test_input_ids),
              torch.tensor(test_attention_masks), 
              torch.tensor(test_labels))
dataloader_test=DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=4
)

AssertionError: ignored

In [134]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_2.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [136]:
_, predictions, true_test = evaluate(dataloader_test)

NameError: ignored

In [135]:
accuracy_per_class(predictions, true_test)

NameError: ignored

In [ ]:
print(classification_report_func(predictions, true_test))